# Applying a Cloud Mask to a Single Image
### ECOSTRESS Tutorials
###### This code is best used for instances when you have a single image and its associated cloud mask file that you want to be applied. If you want multiple images to be cloud masked at once, see our Batch Cloud Mask code.
###### This code is written to cloud mask a Land Surface Temperature (LST) file, but may be modified for use with other ECOSTRESS data products.

### Import the Libraries we Need to Apply the Cloud Mask

In [ ]:
#Import the libraries we need to apply the cloud mask
from os.path import join, basename, splitext
from datetime import datetime
import numpy as np
import rioxarray
import hvplot.xarray
import rasterio

### Load and View the Surface Temperature Image

In [ ]:
#Replace this with the path to your LST image, wrapped in quotes
ST_filepath = r"Replace_this_text_with_file_path"
#Gets the date from the file name
datetime_UTC = datetime.strptime(splitext(basename(ST_filepath))[0].split("_")[-2][3:], "%Y%j%H%M%S")
#Opens the file as a raster image
ST = rioxarray.open_rasterio(ST_filepath).squeeze("band", drop=True)
#Replaces 0s with NaN, then multiplies by the scale factor to get temperature in Kelvin
#Then, to covert from Kelvin to Celcius, we subtract 273.15
ST.data = np.where(ST.data == 0, np.nan, ST.data * 0.02) - 273.15
#rio.reproject reprojects the data into the desired coordinate system
#hvplot.image plots the image with the following parameters
ST.rio.reproject("EPSG:3857").hvplot.image(
    cmap="jet", #Uses the jet colormap
    tiles="OSM", #Uses Open Street Map as the background
    alpha=0.7, #Sets the transparency of the image to 70%
    #Gives it a title based on the date
    title = f"ECOSTRESS Surface Temperature {datetime_UTC:%Y-%m-%d %H:%M:%S} UTC Un-Masked",
    width=1080, #defines the size of the plot
    height=720
)

### Load and View the Cloud Mask

In [ ]:
#Replace this with the folder name of where the files are kept, wrapped in quotes
#Make sure the date of your cloud mask matches the date of your ST file
cloud_filepath = r"Replce_this_text_with_file_path"
#Opens the file as a raster image
cloud = rioxarray.open_rasterio(cloud_filepath).squeeze("band", drop=True).rio.reproject_match(ST)
#Bitshifts the array two bits to the right, and uses & 1 to create a binary cloud mask
cloud.data = (cloud.data >> 2) & 1
#rio.reproject reprojects the data into the desired coordinate system
#hvplot.image plots the image with the following parameters
cloud.rio.reproject("EPSG:3857").hvplot.image(
    cmap="jet", #Uses the jet color map. Red will show cloud areas that need to be masked out
    tiles="OSM", #Uses Open Street Map as the background
    alpha=0.7, #Sets the transparency of the image to 70%
    #Gives it a title based on the date
    title = f"ECOSTRESS Cloud Mask {datetime_UTC:%Y-%m-%d %H:%M:%S} UTC",
    width=1080, #defines the size of the plot
    height=720
)

### Apply the Cloud Mask to the Surface Temperature Image

In [ ]:
#If the cloud mask is true, set the pixel to NaN 
ST.data = np.where(cloud.data, np.nan, ST.data)
#rio.reproject reprojects the data into the desired coordinate system
ST_reprojected = ST.rio.reproject("EPSG:3857")
#hvplot.image plots the image with the following parameters
ST_reprojected.hvplot.image(
    cmap="jet", #Uses the jet color map
    tiles="OSM", #Uses Open Street Map as the background
    alpha=0.7, #Sets the transparency of the image to 70%
    #Gives it a title based on the date
    title = f"ECOSTRESS Surface Temperature {datetime_UTC:%Y-%m-%d %H:%M:%S} UTC Masked",
    width=1080, #defines the size of the plot
    height=720
)

### Save the Masked Image

In [ ]:
#Fill in the variables below with the location of your files
#Replace this with the folder name of where you want the output files to be saved, wrapped in quotes
output_folder_path = r"Replace_this_text_with_folder_path" 
#Replace this with the file name that you want the masked image to be saved as, wrapped in quotes and endig in .tif
output_file_name = f"ECOSTRESS_ST_{datetime_UTC:%Y-%m-%d %H:%M:%S}_Masked.tif"
#Joins the file name with its desired location
output_file_path = join(output_folder_path , output_file_name)

#Get the metadata from the original ST file
with rasterio.open(ST_filepath) as src:
    out_meta = src.meta.copy()

#Update the metadata without applying the scale factor again
out_meta.update({
    "driver": "GTiff",
    "height": ST.shape[0],
    "width": ST.shape[1],
     "dtype": 'float32'
})
    
#Save the masked image with the updated metadata
with rasterio.open(output_file_path, 'w', **out_meta) as output_raster:
    output_raster.write(ST.data.astype('float32'), 1)